In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
pip install causallib
pip install mlens

In [ ]:
import pandas as pd 
import numpy as np 
from scipy.special import expit
from scipy.special import logit
import xgboost as xgb
import statsmodels.api as sm

In [ ]:
# Simulate Data
def simu_data(n, family = "gaussian"):
    W1 = np.random.normal(0, 1, n)
    W2 = np.random.uniform(0, 3, n)
    W3 = np.random.binomial(1, 0.5, n)

    fa = expit(W1 + W2*W3)
    A = np.random.binomial(1, fa)

    Y = expit(W1 + 1.5*W2 + W1*W2*A + 3*W3*A + 1.5)
    Y0 = expit(W1 + 1.5*W2 + 1.5)
    Y1 = expit(W1 + 1.5*W2 + W1*W2+ 3*W3 + 1.5)
    # fy = expit(W1 + W2 + W1*W3*A)
    # Y = 1*(fy >0.5)
    # fy0 = expit(W1 + W2)
    # Y0 = 1*(fy0 >0.5)
    # fy1 = expit(W1 + W2 + W1*W3)
    # Y1 = 1*(fy1 >0.5)
    df = np.column_stack((Y, Y1, Y0, A, W1, W2, W3))
    return(df)
    
def simu_data2 (n):
    w1 = np.random.binomial(1, 0.5, n)
    w2 = np.random.normal(0, 1, n)
    w3 = np.random.uniform(0, 4, n)
    w4 = np.random.uniform(0, 5, n)

    fa = expit(-0.4 + 0.2*w2 + 0.15*w3 + 0.2*w4 + 0.15*w2*w4)
    # fy1 = expit(-1 + 1 -0.1*w1 + 0.3*w2 + 0.25*w3 + 0.2*w4 + 0.15*w2*w4)
    # fy0 = expit(-1 + 0 -0.1*w1 + 0.3*w2 + 0.25*w3 + 0.2*w4 + 0.15*w2*w4)
    fy1 = expit(-0.1*w1 + np.sin(0.3*w2) + 0.25*w3 + 0.2*w4 + 0.15*w2*w4)
    fy0 = expit(np.sin(0.3*w2) + 0.2*w4 + 0.15*w2*w4)

    A = np.random.binomial(1, fa)
    Y1 = np.random.binomial(1, fy1)
    Y0 = np.random.binomial(1, fy0)
    Y = A*Y1 + (1 - A)*Y0

    df = np.column_stack((Y, Y1, Y0, A, w1, w2, w3, w4))
    return(df)

In [ ]:
# continuous Y
df = simu_data(400)
W = df[:,4:]
WA = df[:,3:]
Y = df[:, 0]
A = df[:,3]

# true EY1 (or reward under arm 1) 0.93
np.mean(df[:, 1])

In [ ]:
# discrete Y
df = simu_data2(200)
W = df[:,4:]
WA = df[:,3:]
Y = df[:, 0]
A = df[:,3]

# true EY1 (or reward under arm 1) 0.696
np.mean(df[:, 1]) 

0.685

In [ ]:
def est_f (df):

  W = df[:,4:]
  WA = df[:,3:]
  Y = df[:, 0]
  A = df[:,3]

  # fit Q, g
  xgb_Q = xgb.XGBClassifier(objective="binary:logistic")
  xgb_Q.fit(WA, Y)

  xgb_g = xgb.XGBClassifier(objective="binary:logistic")
  xgb_g.fit(W, A)

  # pred Q
  WA_1 = np.copy(WA)
  WA_1[:,0] = 1
  WA_0 = np.copy(WA)
  WA_0[:,0] = 0

  Q1n = xgb_Q.predict_proba(WA_1)[:,1]
  Q0n = xgb_Q.predict_proba(WA_0)[:,1]
  Qan = A*Q1n + (1-A)*Q0n

  # pred g
  gn = xgb_g.predict_proba(W)[:,1]

  # DM
  psi_dm = np.mean(Q1n)

  # AIPW
  psi_aipw = np.mean((Y - Q1n) * A/gn + Q1n)

  # logistic update
  # gn_c = np.clip(gn, 0.025, 0.975)
  gn_c = gn
  H1n = 1/gn_c
  H0n = 1/(1-gn_c)
  Han = A*H1n + (1-A)*np.array([0]*len(A))

  logLFM = sm.GLM(Y, 
                  Han, 
                  offset = logit(Qan), 
                  family = sm.families.Binomial()).fit()
  eps = logLFM.params.item()
  Q1n_star = expit(logit(Q1n) + eps*H1n)

  # TMLE
  psi_tmle = np.mean(Q1n_star)
  return [psi_dm, psi_aipw, psi_tmle]


In [ ]:
df = simu_data2(200)
est_f(df)

[0.73786443, 0.745236352322961, 0.7378543]

In [ ]:
res_list = []
for i in range(500):
  print("task ", i, " done")
  df = simu_data2(1000)
  res = est_f(df)
  res_list.append(res)

In [ ]:
res_np = np.array(res_list)

In [ ]:
simu_rounds = np.linspace(1, 500, 500)
psi_dm = res_np[:, 0]
psi_aipw = res_np[:, 1]
psi_tmle = res_np[:, 2]

# bias
b1 = np.mean(psi_dm - 0.696)
b2 = np.mean(psi_aipw - 0.696)
b3 = np.mean(psi_tmle - 0.696)

# var
var1 = np.var(psi_dm)
var2 = np.var(psi_aipw)
var3 = np.var(psi_tmle)

# mse
mse1 = b1 **2 + var1
mse2 = b2 **2 + var2
mse3 = b3 **2 + var3

In [ ]:
res_dc = {'Method': ["DM", "AIPW", "TMLE"], 
         'Bias': [b1, b2, b3], 
         'Variance': [var1, var2, var3]}

res_df = pd.DataFrame(data=res_dc)

In [ ]:
print(res_df.to_latex(index=False))  

\begin{tabular}{lrr}
\toprule
Method &      Bias &  Variance \\
\midrule
    DM & -0.007596 &  0.000354 \\
  AIPW & -0.002491 &  0.000377 \\
  TMLE & -0.000704 &  0.000392 \\
\bottomrule
\end{tabular}



In [ ]:
import seaborn as sns
sns.kdeplot(psi_dm, shade= True, color= "g", label="DM")
sns.kdeplot(psi_aipw, shade= True, color= "r", label="AIPW")
sns.kdeplot(psi_tmle, shade= True, color= "b", label="TMLE")
plt.axvline(x = 0.696, color= "black")
plt.legend()
plt.savefig("simu_plot.png")

In [ ]:
# example of a super learner for regression using the mlens library
from math import sqrt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import xgboost as xgb

# gaussian
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from mlens.ensemble import SuperLearner
# binomial
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from mlens.ensemble import SuperLearner
 
# create a list of base-models
def get_models(family):
  models = list()
  if family == "gaussian":
    models.append(LinearRegression())
    models.append(AdaBoostRegressor())
  elif family == "binomial":
    models.append(LogisticRegression(solver='lbfgs'))
    models.append(AdaBoostClassifier())
  return models


# cost function for base models
def rmse(y, yhat):
  return sqrt(mean_squared_error(y, yhat))

def negbinomlik(y, yhat):
  np.mean(Y*np.log(yhat) + (1-Y)*np.log(1-yhat))
  return sqrt(mean_squared_error(y, yhat))
 
# create the super learner
def get_super_learner(X, family):
  if family == "gaussian":
    ensemble = SuperLearner(scorer=rmse, folds=10, shuffle=True, sample_size=len(X))
    # add base models
    models = get_models(family)
    ensemble.add(models)
    # add the meta model
    ensemble.add_meta(LinearRegression())
  elif family == "binomial":
    ensemble = SuperLearner(scorer=accuracy_score, folds=10, shuffle=True, sample_size=len(X))
    # add base models
    models = get_models(family)
    ensemble.add(models)
    # add the meta model
    ensemble.add_meta(LogisticRegression(solver='lbfgs'), proba = True)
  return ensemble

In [ ]:
# sl fit Q, g
sl_Q = get_super_learner(WA, "gaussian")
sl_Q.fit(WA, Y)

sl_g = get_super_learner(W, "binomial")
sl_g.fit(W, A)

# pred Q
WA_1 = np.copy(WA)
WA_1[:,0] = 1
WA_0 = np.copy(WA)
WA_0[:,0] = 0

Q1n = sl_Q.predict(WA_1)
Q0n = sl_Q.predict(WA_0)
Qan = A*Q1n + (1-A)*Q0n

# pred g
gn = sl_g.predict(W)[:,1]

# aipw
aipw = np.mean((Y - Q1n) * A/gn + Q1n)
# aipw = np.mean(Q1n - Q0n + (2*A - 1)/gn * (Y - Qan))
aipw